In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import skimage
import skimage.transform
import skimage.filters
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda
import torch.optim as optim
from torch.nn.functional import relu
from tqdm.notebook import tqdm
from SHG import SHG

In [2]:
def create_heatmaps(keypoints, input_shape = (256, 256), output_shape = (64, 64)):
    """ GIVEN ONE OF THE ROWS IN THE CSV, CREATES THE 17 CORRESPONDING HEATMAPSS """
    keypoints = keypoints[0, 1:] # We dont need the ID of the image

    x_val = keypoints[::3]
    y_val = keypoints[1::3]
    v_val = keypoints[2::3]

    res_arr = []

    for x, y, v in zip(x_val, y_val, v_val):
        x, y, v = int(x), int(y), int(v)
        res = np.zeros(input_shape)

        if (x >= 0 and x <= input_shape[1] and y >= 0 and y <= input_shape[0]):
            res[y - 1, x - 1] = 1

        res = skimage.transform.resize(res, output_shape)
        
        if (v == 1):
            res = skimage.filters.gaussian(res, sigma = 1)
        elif (v == 2):
            res = skimage.filters.gaussian(res, sigma = 0.5)

        res_arr.append(res)

    return res_arr

In [3]:
TRAIN_LABELS_PATH = "D:/bsc_data/train/outputs.txt"
TEST_LABELS_PATH = "D:/bsc_data/test/outputs.txt"
VAL_LABELS_PATH = "D:/bsc_data/validation/outputs.txt"

HEADER = ["ID"]
for i in range(17):
    HEADER.append("x{}".format(i))
    HEADER.append("y{}".format(i))
    HEADER.append("v{}".format(i))

train_labels = pd.read_csv(TRAIN_LABELS_PATH, delimiter = ",", names = HEADER)
test_labels = pd.read_csv(TEST_LABELS_PATH, delimiter = ",", names = HEADER)
val_labels = pd.read_csv(VAL_LABELS_PATH, delimiter = ",", names = HEADER).T

TRAIN_IMGS_PATH = "D:/bsc_data/train/image/"
TEST_IMGS_PATH = "D:/bsc_data/test/image/"
VAL_IMGS_PATH = "D:/bsc_data/validation/image/"

train_imgs = os.listdir(TRAIN_IMGS_PATH)
test_imgs = os.listdir(TEST_IMGS_PATH)
val_imgs = os.listdir(VAL_IMGS_PATH)

In [4]:
LEARNING_RATE = 2.5e-4
NUM_EPOCHS = 100
MINI_BATCH_SIZE = 6
MINI_BATCHES = np.array_split(train_imgs, len(train_imgs)/MINI_BATCH_SIZE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SHG(num_hourglasses=1).to(device)

criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr = LEARNING_RATE)

In [5]:
torch.cuda.empty_cache()

for epoch in range(NUM_EPOCHS):
    for mini_batch in MINI_BATCHES:
        heatmaps = []
        imgs = []

        for img in mini_batch:
            heatmaps.append(create_heatmaps(train_labels.loc[train_labels["ID"] == img[:-4]].to_numpy()))
            imgs.append(plt.imread(TRAIN_IMGS_PATH + img))
        
        heatmaps = torch.FloatTensor(heatmaps).to(device)
        imgs = (torch.from_numpy(np.array(imgs)).permute(0, 3, 1, 2)).to(device)
        
        model(imgs)

        predictions = torch.from_numpy(model.pred[-1]).to(device)
        print(predictions.shape)
        print(heatmaps.shape)

        loss = criterion(predictions, heatmaps)

        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        break
    break
        


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same